In [ ]:
import pandas as pd 
import requests
from dotenv import load_dotenv
import os
import time

import spotipy
from spotipy.oauth2 import SpotifyOAuth
from requests.exceptions import ReadTimeout

from tqdm import tqdm

In [ ]:
# CREDENCIAIS
load_dotenv()

CHAVE_CLIENTE=os.getenv("CHAVE_CLIENTE")
SEGREDO=os.getenv("SEGREDO")
URI=os.getenv("URI")

### Pega todos os 200 artistas de forró mais populares do Spotify

In [ ]:
# USANDO O SPOTIPY PARA CREDENCIAIS

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CHAVE_CLIENTE,
                                               client_secret=SEGREDO,
                                               redirect_uri=URI,
                                               scope="user-library-read"))

In [ ]:
# CRIA UM DATAFRAME A PARTIR DO QUE APARECE

# OFFSET: É COMO UMA PAGINAÇÃO. QUANDO ELE É 0, ELE PEGA OS ARTISTAS DE 0 A 50. QUANDO É 50, PEGA OS PRÓXIMOS 50 (50 A 100)

paginas=[0,50,100,150] # PEGA OS 150 PRIMEIROS ARTISTAS QUE APARECEM NO SPOTIFY PARA PESQUISA "FORRÓ"


dados_completos=pd.DataFrame()

for pagina in paginas:
    print(pagina)
    resultado = sp.search(q='genre:"forro"', type='artist',offset=pagina,limit=50)

    dados=pd.DataFrame()

    for artista in tqdm(resultado['artists']['items'],desc="Artistas"):
       
        nome=(artista['name'])
        foto=(artista['images'][0]['url'])
        generos=', '.join(artista['genres'])
        seguidores=(artista['followers']['total'])
        id=(artista['id'])
        popularidade=(artista['popularity'])

        df=pd.DataFrame({
            "pagina":[pagina], # COLOCA A PÁGINACAO PRA NÃO SE PERDER
            "id":[id],
            "artista":[nome],
            "foto":[foto],
            "generos":[generos],
            "seguidores":[seguidores],
            "popularidade":[popularidade],
        })

        dados=pd.concat([dados,df],ignore_index=True)
    
    dados_completos=pd.concat([dados_completos,dados],ignore_index=True)

### Pega álbum de cada artista

In [ ]:
# PEGA DISCOGRAFIA DE CADA ARTISTA 
# CONSIDERA SOMENTE OS ALBUMS. DESSA FORMA, NAO ENTRAM SINGLES OU PARTICIPAÇÕES

# Lista de IDs únicos dos artistas
ids_artistas = list(set(dados_completos['id']))  # Remove duplicatas da lista de IDs

# DataFrame para armazenar as informações de álbuns
dados_albums = pd.DataFrame()

for id_artista in tqdm(ids_artistas):
    albums = []
    offset = 0
    limit = 50
    retries = 5  # Número de tentativas

    while True:
        try:
            # Requisição para pegar os álbuns do artista com o offset crescente
            response = sp.artist_albums(id_artista, album_type="album", limit=limit, offset=offset)
            albums += response['items']  # Adiciona os álbuns retornados à lista

            # Verifica se há mais álbuns para pegar
            if len(response['items']) < limit:
                break
            offset += limit  # Atualiza o offset para pegar os próximos álbuns
            
            time.sleep(1)  # Espera 1 segundo antes da próxima requisição para não sobrecarregar a API

        except ReadTimeout:
            retries -= 1
            if retries == 0:
                print(f"Erro de timeout repetido para o artista {id_artista}. Abortando...")
                break
            print(f"Timeout ocorreu para o artista {id_artista}. Tentando novamente... ({5 - retries} tentativas restantes)")
            time.sleep(2)  # Espera 2 segundos antes de tentar novamente

    # Processa as informações dos álbuns coletados
    for album in albums:
        # Garante que o artista principal do álbum corresponde ao ID atual
        album_artist_ids = [artist['id'] for artist in album['artists']]
        if id_artista not in album_artist_ids:
            continue  # Ignora álbuns que não pertencem diretamente ao artista

        # Cria o dicionário com os dados do álbum
        album_data = {
            "id_artista": id_artista,
            "nome_artista": [artist['name'] for artist in album['artists'] if artist['id'] == id_artista][0],
            "id_album": album['id'],  # ID do álbum
            "nome_album": album['name'],  # Nome do álbum
            "total_faixas": album['total_tracks'],  # Nome do álbum
            "album_type": album['album_type'],  # Tipo do álbum
            "release_date": album.get('release_date', None),  # Data de lançamento
            "album_group": album.get('album_group', None),  # Grupo do álbum (se disponível)
            "capa_album": album['images'][0]['url'] if album['images'] else None,  # URL da capa (maior resolução)
            "artistas_album": ", ".join([artist['name'] for artist in album['artists']])  # Lista de artistas como string
        }

        dados_albums = pd.concat([dados_albums, pd.DataFrame([album_data])], ignore_index=True)

# Remove duplicatas no DataFrame final (caso existam)
dados_albums.drop_duplicates(subset=["id_album"], inplace=True)

# Exibe os primeiros registros
print(len(dados_completos['id'].unique()))  # 200 COM ID ÚNICA
print(len(dados_albums['nome_artista'].unique()))  # MENOS DE 200 (NUNCA MAIS DE 200)

completo=list(dados_completos['artista'].unique())
albums=list(dados_albums['nome_artista'].unique())

diferenca=[x for x in completo if x not in albums]
print(diferenca)